In [2]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
import json

In [17]:
loc = 'India'
pars_cs = pd.read_csv(f'docs/tabs/post_cs_nods_{loc}.csv')
pars_cs = [dict(row) for _, row in pars_cs.iterrows()]
pars_txi = pd.read_csv('docs/tabs/post_txi.csv').set_index('State')

In [28]:
class I:
    NonTB = 0
    Asym = 1
    Sym = 2
    ExCS = 3
    TxPub = 4
    TxEngPu = 5
    TxEngPr = 6
    TxPri = 7

In [21]:
p = pars_cs[0]
p.keys()

dict_keys(['prv0', 'adr', 'p_pub', 'p_ppm', 'r_sym', 'r_aware', 'r_acf', 'r_det_all', 'r_sc', 'txi_pri', 'ppv_pri', 'dur_pri', 'p_pri_on_pub', 'ra', 'rs', 'rc', 'r_det', 'a0', 'c0', 'pr_a', 'pr_s', 'pr_c', 'prv_a', 'prv_s', 'prv_c', 'prv_t_eu', 'prv_t_ei', 'prv_t_pri', 'det_pub', 'det_eng', 'det_pri', 'scale_dur', 'r_death_a', 'r_death_s', 'ppv_pub', 'ppv_eng', 'sens_acf', 'spec_acf', 'dur_pub'])

In [35]:
pars_txi.loc[loc, ]

p_txi_pub_m    0.941428
p_txi_pub_l    0.941215
p_txi_pub_u    0.941647
p_txi_eng_m    0.884097
p_txi_eng_l    0.883616
p_txi_eng_u    0.884574
Name: India, dtype: float64

In [30]:
year0 = 2015

k = np.exp(- p['adr'] * (year0 - 2020))
prev = k * p['prv0']

y0 = np.zeros(8)
y0[I.NonTB] = 1 - prev
y0[I.Asym] = p['prv_a'] * k
y0[I.Sym] = p['prv_s'] * k
y0[I.ExCS] = p['prv_c'] * k

y0.sum()


0.9999999999999998

In [76]:
def fn(t, y, pars):
    onset = pars['r_sym'] *  y[I.Asym]
    aware = pars['r_aware'] * y[I.Sym]
    
    p_det = np.array([pars['p_pub'], (1 - pars['p_pub']) * pars['p_ppm'], (1 - pars['p_pub']) * (1 - pars['p_ppm'])])
    # p_txi = np.array([pars['txi_pub'], pars['txi_eng'], pars['txi_pri']])
    p_txi = np.array([0.94, 0.88, pars['txi_pri']])
    det = pars['r_det'] * p_det * p_txi * y[I.ExCS]
    
    sc_a = pars['r_sc'] * y[I.Asym]
    sc_s = pars['r_sc'] * y[I.Sym]
    sc_c = pars['r_sc'] * y[I.ExCS]
    die_a = pars['r_death_a'] * y[I.Asym]
    die_s = pars['r_death_s'] * y[I.Sym]
    die_c = pars['r_death_s'] * y[I.ExCS]
    
    acf_a = pars['r_acf'] * pars['sens_acf'] * y[I.Asym]
    acf_s = pars['r_acf'] * pars['sens_acf'] * y[I.Sym]
    acf_c = pars['r_acf'] * pars['sens_acf'] * y[I.ExCS]
    
    acf_a = acf_s = acf_c = 0
    
    
    inc = onset + acf_a + sc_a + die_a - pars['adr'] * y[I.Asym]
    
    dy = np.zeros_like(y)
    dy[I.Asym] += inc - onset - acf_a - sc_a - die_a
    dy[I.Sym] += onset - aware - acf_s - sc_s - die_s
    dy[I.ExCS] += aware - det.sum() - acf_c - sc_c - die_c
    return dy
    
    

In [77]:
ys = solve_ivp(fn, [2015, 2025], y0 = y0, args = (p, ))

In [78]:
np.diff(np.log(ys.y[I.Asym]))/ np.diff(ys.t)

array([-0.0453766, -0.0453766, -0.0453766, -0.0453766, -0.0453766,
       -0.0453766, -0.0453766, -0.0453766, -0.0453766, -0.0453766])

In [79]:
p['r_acf'] * p['sens_acf']

0.0025725450377081